## This notebook exemplifies an use-case of the tool able to extract coordinates and pixel values from a SENTINEL2B photo.

This algorithm can work for every other satellite, but it may need some parameter tweaking, more specifically the parameter that determines the projection type. 

Needed imports

In [1]:
import numpy as np
import cv2
import os, glob
import pandas as pd
from itertools import product
import affine
from affine import Affine
import pyproj
from pyproj import Transformer
import dask.dataframe as dd
import dask.array as da
import matplotlib.pyplot as plt
import rasterio
import rasterio.features
import keplergl
import ast
import configparser

### Declaring directory containing the photos
Store the desired path in the "sat" variable inside the "config.txt" file.

In [2]:
paths = configparser.ConfigParser()
paths.read_file(open(r'config.txt'))
sat_img = paths.get('paths', 'sat')
img_list = glob.glob(os.path.join(sat_img , '*.tif'))

### Variables needed to perform the affine transformation that converts the pixels positions to UTM XY coordinates.


In [3]:
img = rasterio.open(img_list[0])
img_left = img.bounds[0]
img_bottom = img.bounds[1]
img_right = img.bounds[2]
img_top = img.bounds[-1]

### An affine transformation is a geometric transformation that preserves lines and parallelism, and if needed, distances and angles.
The parameters given to the affine tranformation are defined as such:
* Height of a pixel in the north-south direction.
* Angle in the Y axis. 
* UTM Y coordinates of top left pixel.
* Angle in X axis. 
* Width of a pixel in the east-west direction.
* UTM X coordinates of top right pixel.

In [4]:
pixel_transform = affine.Affine(
        -20,
        0,
        img_top,
        0,
        20,
       img_left    
        )

### Variable that stores which coordinate transformation should be done. In this case EPSG 32723 are converted to EPSG 4326


In [5]:
transformer = Transformer.from_crs(32723, 4326,  always_xy =True)

### Converting and storing points.

In [9]:
df_list = []
coords_list = []
cart_list = []
def points():
    for img in img_list:
        im = cv2.imread(img, -1)
        height = range(im.shape[0])
        width = range(im.shape[1])
        im = im.flatten()
        df = pd.DataFrame(im)
        df.columns = ['value']
        df_list.append(df)

        #     height, width = im.shape[:2]

        #     center = (width/2, height/2)

        #     print(width, height)

        #     rotate_matrix = cv2.getRotationMatrix2D(center=center, angle=90, scale=1)

        #     rotated_image = cv2.warpAffine(src=im, M=rotate_matrix, dsize=(width, height))
        coords = np.array(list(product(height, width)))

        #     y = range(rotated_image.shape[0])
        #     x = range(rotated_image.shape[1])
        #     coords = np.array(list(product(y, x)))
        #     print(coords)


        coords_df = pd.DataFrame(coords, columns = ['Y', 'X'])


        yy, xx = pixel_transform * (coords_df['Y'], coords_df['X'])


        xx_latlng, yy_latlng = transformer.transform(xx,yy)

        xx_latlng = pd.Series(xx_latlng)
        yy_latlng = pd.Series(yy_latlng)

        xx_yy = pd.concat([xx_latlng, yy_latlng], axis=1)
        
        xx_yy.columns = ['Longitude', 'Latitude']
        
        name = img.split('\\')
        name = name[-1]
        name = name.replace('.tif', '')
        
        xx_yy.name = name
        df.name = name

        cart_list.append(coords_df)

        coords_list.append(xx_yy)
points()

minimal = df_list[0]['value'].min()
path_namer = sat_img
df_namer = glob.glob(os.path.join(path_namer , '*.tif'))

### The coordinates and pixel values obtained above are stored in a .csv file in a folder named data_sat.
The folder creates itself if it doesn't already exists.

In [10]:
points = []

if os.path.isdir("sat_data"):
    print('Diretório sat_data já existe')
else:
    print('Diretório sat_data criado')
    os.mkdir("sat_data")

for d in df_list:
    print(d.name)
    merge = pd.concat([d, coords_list[0]], axis=1)
    merge_mask = merge[(merge['value'] > minimal)]
    img_band = df_namer[0].replace(path_namer, '')
    img_band = img_band.replace('_20m_Mask_NDWI.tif', '')
    img_band = img_band.replace('\\', '')
    merge_mask.name = img_band
    print(merge_mask.name)
    df_namer.pop(0)
    coords_list.pop(0)
    #     img_band = df_namer[0].replace(path_namer, '')
    #     img_band = img_band.replace('_20m_Mask_NDWI.tif', '')
    #     merge.name = img_band
    #coords_list.pop(0)
    #    df_namer.pop(0)

    points.append(merge_mask)
for j in points:
    j.to_csv('sat_data/{}.csv'.format(j.name), index=False)

Diretório sat_data já existe
B2_20m_Mask_NDWI
B2
B3_20m_Mask_NDWI
B3
B4_20m_Mask_NDWI
B4
